In [41]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score

In [2]:
master_data = pd.read_csv('seasonal_clustering_data_master.csv', index_col=[0, 1])
master_data

MP/G       GS%       PS/G      FG/G  \
season_end player                                                      
2001       A.C. Green       17.207317  0.012195   4.475610  1.756098   
           A.J. Guyton      19.090909  0.242424   6.000000  2.363636   
           Aaron McKie      31.500000  0.434211  11.552632  4.447368   
           Aaron Williams   28.487805  0.304878  10.219512  3.621951   
           Adam Keefe       12.477612  0.194030   2.507463  0.955224   
...                               ...       ...        ...       ...   
2024       Zach Collins     26.129032  0.741935  12.935484  4.935484   
           Zach LaVine      35.347826  0.913043  20.086957  6.956522   
           Zeke Nnaji        9.647059  0.000000   3.235294  1.176471   
           Ziaire Williams  18.918919  0.216216   8.027027  2.864865   
           Zion Williamson  30.375000  1.000000  21.968750  8.656250   

                                  FGA       FG%      2P/G      2PA/G  \
season_end player                                                      
2001       A.C. Green        3.951220  0.444444  1.756098   3.878049   
           A.J. Guyton       5.818182  0.406250  1.545455   3.727273   
           Aaron McKie       9.394737  0.473389  3.750000   7.157895   
           Aaron Williams    7.926829  0.456923  3.621951   7.902439   
           Adam Keefe        2.373134  0.402516  0.940299   2.328358   
...                               ...       ...       ...        ...   
2024       Zach Collins      9.967742  0.495146  3.838710   6.451613   
           Zach LaVine      15.434783  0.450704  4.565217   8.478261   
           Zeke Nnaji        2.470588  0.476190  1.029412   2.029412   
           Ziaire Williams   7.054054  0.406130  1.675676   3.540541   
           Zion Williamson  14.812500  0.584388  8.562500  14.531250   

                                 2P%      3P/G  ...  Left Corner 3_shots_pct  \
season_end player                               ...                            
2001       A.C. Green       0.452830  0.000000  ...                      NaN   
           A.J. Guyton      0.414634  0.818182  ...                 0.363636   
           Aaron McKie      0.523897  0.697368  ...                 0.333333   
           Aaron Williams   0.458333  0.000000  ...                      NaN   
           Adam Keefe       0.403846  0.014925  ...                 0.000000   
...                              ...       ...  ...                      ...   
2024       Zach Collins     0.595000  1.096774  ...                 0.222222   
           Zach LaVine      0.538462  2.391304  ...                 0.529412   
           Zeke Nnaji       0.507246  0.147059  ...                 0.000000   
           Ziaire Williams  0.473282  1.189189  ...                 0.310345   
           Zion Williamson  0.589247  0.093750  ...                      NaN   

                            Mid-Range_shot_made  Mid-Range_shots_attempted  \
season_end player                                                            
2001       A.C. Green                      80.0                      194.0   
           A.J. Guyton                     37.0                       99.0   
           Aaron McKie                    118.0                      266.0   
           Aaron Williams                  44.0                      151.0   
           Adam Keefe                      19.0                       68.0   
...                                         ...                        ...   
2024       Zach Collins                    10.0                       26.0   
           Zach LaVine                     30.0                       63.0   
           Zeke Nnaji                       0.0                        1.0   
           Ziaire Williams                 11.0                       33.0   
           Zion Williamson                  1.0                        8.0   

                            Mid-Range_shots_pct  Restricted Area_shot_made  \
season_end player        

In [ ]:
clustering_data = master_data
clustering_data['']

In [44]:
seasonal_data = clustering_data.groupby('season_end')
max_average_silhouette = 0
max_average_calinski_harabasz = 0
k_dict = {}
for k in tqdm(range(5, 35)):
    total_silhouette = 0
    total_calinski_harabasz = 0
    for group in seasonal_data:
        X = group[1].reset_index(drop=True)
        X.dropna(inplace=True)
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(X)

        # pca = PCA(n_components=0.9)
        # pca_data = pca.fit_transform(scaled_data)

        labels = KMeans(n_clusters=k, random_state=42, n_init=10).fit(scaled_data)
        labels = labels.labels_

        total_calinski_harabasz += calinski_harabasz_score(scaled_data, labels)
        total_silhouette += silhouette_score(scaled_data, labels)

    average_silhouette = total_silhouette / len(seasonal_data)
    average_calinski_harabasz = total_calinski_harabasz / len(seasonal_data)

    # Update max averages if current averages are higher
    if average_silhouette > max_average_silhouette:
        max_average_silhouette = average_silhouette
    if average_calinski_harabasz > max_average_calinski_harabasz:
        max_average_calinski_harabasz = average_calinski_harabasz

    k_dict[k] = (total_silhouette / len(seasonal_data), total_calinski_harabasz / len(seasonal_data))
k_dict

  0%|          | 0/30 [00:00<?, ?it/s]

{5: (0.13260421259183658, 25.23791490322901),
 6: (0.12192006729393041, 22.85470544339223),
 7: (0.11847760069636865, 20.937378674854102),
 8: (0.11154154520893005, 19.520467980627),
 9: (0.10613335384293349, 18.27817492203007),
 10: (0.10502569483253421, 17.232786702328145),
 11: (0.10186315145783369, 16.32577114678523),
 12: (0.09790554217506972, 15.624721176482879),
 13: (0.09679931602612425, 14.987693034904726),
 14: (0.09271403078094333, 14.353251431771913),
 15: (0.09403096511374388, 13.93426066721734),
 16: (0.09407146344250582, 13.44740167531263),
 17: (0.09350889810489098, 13.011851850437534),
 18: (0.09315007092305075, 12.566405404757935),
 19: (0.08931928936477615, 12.282383033216277),
 20: (0.09007235974018885, 11.996659361948103),
 21: (0.08857599673333198, 11.748245505016897),
 22: (0.08841159171341574, 11.456612271149993),
 23: (0.08734953848484082, 11.235340327803652),
 24: (0.08747393332229288, 11.041726019571044),
 25: (0.08240832120728639, 10.835370845394895),
 26: (